<a href="https://colab.research.google.com/github/gabriel-alk/Automatizacao-de-analise-de-mensagens-de-um-grupo-do-whatsapp/blob/main/An%C3%A1liseSFT_rev.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Introdução

O código foi desenvolvido em Python com o objetivo de automatizar o processo de contagem dos dias treinados por cada membro de um grupo do whatsapp, a partir da análise do arquivo .txt gerado pelo aplicativo ao exportar uma conversa. Para mais detalhes sobre o grupo leia o arquivo README.md: https://github.com/gabriel-alk/Automatizacao-de-analise-de-mensagens-de-um-grupo-do-whatsapp/blob/main/README.md

Primeiramente o código faz a leitura do arquivo de texto gerado pela exportação da conversa do WhatsApp. O usuário então deve inserir o mês que deseja fazer a verificação da pontuação.

A identificação do envio das fotos foi feita por meio da procura por mensagens contendo a cadeia de texto <Arquivo de mídia oculto>, gerada no arquivo txt quando se exportam conversas sem incluir arquivos de mídia. 

Após a identificação das mensagens com a cadeia de texto, o código faz a extração da data da mensagem e contagem de quantas mensagens foram enviadas por cada membro em no mês selecionado. 

Para realizar a análise, o código cria um dicionário com o nome de cada membro do grupo e, a cada mensagem contendo a cadeia de texto de identificação de arquivo de mídia encontrada, adiciona +1 na contagem da pontuação para o respectivo membro. Ao final do processo, o código ordena o dicionário de acordo com a pontuação de cada membro, mostrando o ranking final com a quantidade de mensagens enviadas por cada um, na forma de "X° lugar: Nome do usuário Y pontos".

Em caso do envio de mensagens não válidas para a pontuação, o usuário deve enviar logo em seguida uma mensagem contendo a cadeia de texto "invalidar", que é interpretada pelo código e não computa na pontuação do usuário.

O código também exporta o resultado da análise em um arquivo .xlsx que é disponibilizado para os usuários checarem os dias em que pontuaram. 


In [1]:
import pandas as pd

url = '/content/drive/MyDrive/SFT_teste3.txt' #link para acesso ao arquivo txt.

print('Insira qual o mês da medição no formato mm/aaaa: ') #O usuário deve inserir o mês que deseja verificar
periodo_medicao = input()

Insira qual o mês da medição no formato mm/aaaa: 
12/2021


In [2]:
# Inicializa o dicionário para armazenar a contagem de cada usuário
contagem = {}

# Abre o arquivo em modo de leitura
with open(url, "r") as arq: 
    
    linhas = arq.readlines()  # Lê todas as linhas do arquivo    
    target_string = "<Arquivo de mídia oculto>" # Define a cadeia de texto a ser contada
    invalid_string = "invalidar" #cadeia de texto para invalidar a pontuação de uma foto não válida
    last_user = "" #nome do ultimo usuario que enviou a mensagem
    
    for i, linha in enumerate(linhas):
        
        if linha[3:10]==periodo_medicao: #if para verificar apenas o mes selecionado
          
          if target_string in linha and not (i+1 < len(linhas) and invalid_string in linhas[i+1]): # Verifica se a linha contém a cadeia de texto de arquivo de midia e se a próxima linha não é para invalidar
              
              palavras = linha.split()  # Separa a linha (string) em uma lista de palavras
              nome_usuario = linha.split(" - ")[1].split(":")[0] #atribui parte da cadeia de texto a variavel do nome do usuario
              data_ponto = linha.split(' ')[0] #atribui parte da cadeia de texto a variavel da data da pontuação

              if nome_usuario in contagem: #verifica se o nome do usuario ja esta na lista
                  if data_ponto in contagem[nome_usuario]: 
                      contagem[nome_usuario][data_ponto] += 1 #soma +1 na pontuação se ja estiver
                  else:
                      contagem[nome_usuario][data_ponto] = 1 #se não estiver salva como 1 a pontuação do usuário
              else:
                  contagem[nome_usuario] = {data_ponto: 1} 


In [4]:
# Convertendo o dicionário de contagem para um dataframe do pandas
df = pd.DataFrame.from_dict(contagem, orient='index')

# Ordenando o dataframe pelo índice (datas)
df = df.sort_index(axis=1)

# Salvando o dataframe em um arquivo excel
df.to_excel('resultado_contagem.xlsx')

pontuacao = {} #dicionario para ordernar os participantes por ordem da maior para menor pontuação

#calculando a soma de pontos para cada usuário
for usuario, dias in contagem.items():
    pontuacao[usuario] = sum(dias.values())

# Ordenando o dicionário por valores
ranking = sorted(pontuacao.items(), key=lambda x: x[1], reverse=True)

# Imprimindo o ranking
for i, (usuario, pontos) in enumerate(ranking, start=1):
    print(f"{i}° lugar: {usuario} {pontos} pontos")

1° lugar: Gabriel Mendes 3 pontos
2° lugar: Garibe 2 pontos
3° lugar: Pedroca 2 pontos
4° lugar: Landim 2 pontos
5° lugar: Carol Drummond 2 pontos
6° lugar: Bruno Hauck 1 pontos
7° lugar: Victor Sávio 1 pontos
8° lugar: Simonneti 1 pontos
